In [1]:
import os
import pandas as pd
import re
import backoff
import time
import json
from openai.error import RateLimitError
from importlib.metadata import version
from typing import List, Callable, Dict
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.serpapi import SerpAPIWrapper
from transformers import GPT2TokenizerFast
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
os.environ["OPENAI_API_KEY"] = OAI_KEY
os.environ["SERPAPI_API_KEY"] = SERP_KEY
version('langchain')

'0.0.55'

In [3]:
d = pd.read_csv(FP + "legislation_relevance_dataset_for_llm_evaluation_unbalanced.csv")

In [4]:
print(d.shape[0])
d = d.drop_duplicates(subset=['company_name', 'summary_text'], keep='first')
print(d.shape[0])
d = d.reset_index(drop=True)

500
485


In [5]:
d.relevance.value_counts()

0    344
1    141
Name: relevance, dtype: int64

In [6]:
class Lobbyist:
    def __init__(self, data: pd.DataFrame):
        self.d = data
        self.model_name = "text-davinci-003"
        self.model_name_older = "text-davinci-002"
        self.temp = 0
        
        self.tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
        self.text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=4000, 
                                                                        chunk_overlap=0, 
                                                                        separator = " ")
        self.llm_summarize = OpenAI(temperature=self.temp, model_name=self.model_name)
        self.llm = OpenAI(temperature=self.temp, model_name=self.model_name, max_tokens=-1)
        self.llm_older = OpenAI(temperature=self.temp, model_name=self.model_name_older, max_tokens=-1)
        
        ###### SUMMARY CHAIN ######
        self.sum_template = """Slightly summarize the following Congressional bill summary:
                                {context}
                                SUMMARY:"""
        self.sum_prompt = PromptTemplate(template=self.sum_template, input_variables=["context"])
        self.sum_combine_template = """Given the following summaries of parts of this bill, create a final summary of the bill. 
                                        =========
                                        SUMMARIES FROM PARTS OF THE OVERALL RESPONSE: {summaries}
                                        =========
                                        FINAL SUMMARY:"""
        self.sum_combine_prompt = PromptTemplate(template=self.sum_combine_template, 
                                                 input_variables=["summaries"])
        self.sum_chain = load_qa_chain(llm=self.llm_summarize, 
                                        chain_type="map_reduce", 
                                        question_prompt = self.sum_prompt,
                                        combine_prompt = self.sum_combine_prompt,
                                        collapse_prompt = self.sum_combine_prompt)
        
        ###### ZERO SHOT QA CHAIN ######
        self.zero_shot_template = """You are a lobbyist analyzing Congressional bills for their potential impacts on companies. 
                            Given the title and summary of the bill, plus information on the company from its 10K SEC filing, it is your job to determine if a bill is at least somewhat relevant to a company (in terms of whether it could impact the company if it was later enacted). 
                            Official title of bill: {official_title}
                            Official summary of bill: {summary_text}
                            Official subjects of bill: {subjects}
                            Company name: {company_name}
                            Company business description: {business_description}
                            Is this bill potentially relevant to this company? 
                            Answer in this format:
                            ANSWER: 'YES' or 'NO' (use all caps). EXPLANATION: the step-by-step reasoning you undertook to formulate a response. CONFIDENCE: integer between 0 and 100 for your estimate of confidence in your answer (1 is low confidence and 99 is high)
                            """
        self.zero_shot_temp  = PromptTemplate(template=self.zero_shot_template, input_variables = ["official_title", "summary_text", "subjects",
                                                                     "company_name", "business_description"])
        self.zero_shot_chain = LLMChain(llm=self.llm, prompt=self.zero_shot_temp)
        
        self.zero_shot_chain_older = LLMChain(llm=self.llm_older, prompt=self.zero_shot_temp)
        
        
        
        ###### ZERO SHOT LETTER GENERATION ######
        self.zero_shot_letter_template = """You are a lobbyist analyzing Congressional bills for their impacts on companies. 
                            You have identified the following bill to be relevant to the company you work for.
                            Given the title and summary of the bill, plus information on your company from its 10K SEC filing, it is your job to now write a persuasive letter to sponsor of the bill to convince them to add provisions to the bill that would make it better for the bottom-line of your company.
                            Sign the letter as the general counsel of your company (put the actual name of your company).
                            Official title of bill: {official_title}
                            Official summary of bill: {summary_text}
                            Official subjects of bill: {subjects}
                            Company name: {company_name}
                            Company business description: {business_description}
                            YOUR LETTER: 
                            """
        self.zero_shot_letter_temp  = PromptTemplate(template=self.zero_shot_letter_template, 
                                              input_variables = ["official_title", "summary_text", "subjects",
                                                                "company_name", "business_description"])
        self.zero_shot_letter = LLMChain(llm=self.llm, prompt=self.zero_shot_letter_temp)
    
    def compute_length(self, text: str) -> int:
        tokenized_text = self.tokenizer.tokenize(text)
        nt = len(tokenized_text)
        return nt

    def compute_length_shorten(self, text: str) -> str:
        nt = self.compute_length(text)
        while nt > 4000:
            texts = self.text_splitter.split_text(text)
            docs = [Document(page_content=t) for t in texts]
            text = self.sum_chain.run(docs)
            nt = self.compute_length(text)
        return text

    def add_summary_col(self,):
        results = list()
        for i in range(self.d.shape[0]):
            s = self.compute_length_shorten(self.d.summary_text[i])
            results.append({"summary_text_summarized": s})
        self.d = pd.concat([self.d, pd.DataFrame(results)], axis=1)
    
    def convert_output(self, text: str) -> Dict:
        _out = dict()
        if "YES" in text:
            _out['prediction'] = 1
        elif "NO" in text:
            _out['prediction'] = 0
        elif "Yes" in text:
            _out['prediction'] = 1
        elif "No" in text:
            _out['prediction'] = 0
        elif "yes" in text:
            _out['prediction'] = 1
        elif "no" in text:
            _out['prediction'] = 0
        else:
            _out['prediction'] = -999
        
        match = re.search('CONFIDENCE: (\d+)', text)
        if match:
            _out['prob'] = int(match.group(1))
        else:
            _out['prob'] = -999
        
        return _out

    def dummy_predictor(self, i: int) -> str:
        return "NO and 100"
    
    @backoff.on_exception(backoff.expo, (RateLimitError))
    def zero_shot_predictor(self, i: int) -> str:
        return self.zero_shot_chain.predict(official_title = self.d.official_title[i], 
                                            summary_text = self.d.summary_text_summarized[i], 
                                            subjects = self.d.subjects[i], 
                                            company_name = self.d.company_name[i], 
                                            business_description = self.d.business_description[i])
    
    @backoff.on_exception(backoff.expo, (RateLimitError))
    def zero_shot_predictor_older(self, i: int) -> str:
        return self.zero_shot_chain_older.predict(official_title = self.d.official_title[i], 
                                            summary_text = self.d.summary_text_summarized[i], 
                                            subjects = self.d.subjects[i], 
                                            company_name = self.d.company_name[i], 
                                            business_description = self.d.business_description[i])
    
    
    @backoff.on_exception(backoff.expo, (RateLimitError))
    def zero_shot_letter_generator(self, i: int) -> str:
        return self.zero_shot_letter.predict(official_title = self.d.official_title[i], 
                                            summary_text = self.d.summary_text_summarized[i], 
                                            subjects = self.d.subjects[i], 
                                            company_name = self.d.company_name[i], 
                                            business_description = self.d.business_description[i])
    
    def evaluate(self, func: Callable, 
                 n = int, m = int) -> List:
        results = list()
        for i in range(n, m):
            try:
                out = func(i)
                pred = self.convert_output(out)
                pp = pred['prediction']
                gt = self.d.relevance[i]

                if pp == -999:
                    correct = False
                    print("-999")
                elif pp == 2:
                    prediction = 1
                    correct = gt == pp
                else:
                    correct = gt == pp

                results.append({"text_response": out,
                                "numeric_prediction": pp,
                                "probability": pred['prob'],
                                "ground_truth": int(gt),
                                "correct": bool(correct)})
            except Exception as e:
                print(f"At data {i}, error: {e}")
                time.sleep(10)
                try:
                    out = func(i)
                    pred = self.convert_output(out)
                    pp = pred['prediction']
                    gt = self.d.relevance[i]

                    if pp == -999:
                        correct = False
                        print("-999")
                    elif pp == 2:
                        prediction = 1
                        correct = gt == pp
                    else:
                        correct = gt == pp

                    results.append({"text_response": out,
                                    "numeric_prediction": pp,
                                    "probability": pred['prob'],
                                    "ground_truth": int(gt),
                                    "correct": bool(correct)})
                except Exception as e2:
                    print(f"At data {i}, error 2: {e2}")
                    results.append({"text_response": "NA",
                                    "numeric_prediction": "NA",
                                    "probability": "NA",
                                    "ground_truth": "NA",
                                    "correct": "NA"})
            finally:
                with open(FP + "gpt_data.json", "wt") as output_file:
                    json.dump(results, output_file)
        
        return results

In [7]:
l = Lobbyist(d)
l.add_summary_col()

Token indices sequence length is longer than the specified maximum sequence length for this model (7266 > 1024). Running this sequence through the model will result in indexing errors


In [8]:
dum_result = l.evaluate(l.dummy_predictor, 0, l.d.shape[0])

In [9]:
dum = pd.DataFrame(dum_result)
print(dum.correct.mean())

0.709278350515464


In [10]:
result_chain_older = l.evaluate(l.zero_shot_predictor_older, 0, l.d.shape[0])

In [11]:
result_chain_older = pd.DataFrame(result_chain_older)
result_chain_older.numeric_prediction.value_counts()

1    351
0    134
Name: numeric_prediction, dtype: int64

In [12]:
result_chain_older[['probability', 'numeric_prediction']].value_counts()

probability  numeric_prediction
 80          1                     316
 1           0                      56
-999         0                      38
 99          0                      22
-999         1                      21
 95          1                      11
 80          0                       6
 100         0                       5
 95          0                       3
 60          0                       2
             1                       2
 90          0                       2
 70          1                       1
dtype: int64

In [13]:
print(result_chain_older.text_response[6])



ANSWER: YES
EXPLANATION: The bill is relevant to Alkermes Plc because it could impact the company's products ARISTADA and VIVITROL.
CONFIDENCE: 80


In [14]:
print(result_chain_older.correct.mean())
print(result_chain_older[result_chain_older.ground_truth == 1].correct.mean())
print(result_chain_older[result_chain_older.ground_truth == 0].correct.mean())
result_chain_older_metadata = pd.concat([l.d, result_chain_older], axis=1)
print(precision_score(result_chain_older_metadata.ground_truth, result_chain_older_metadata.numeric_prediction))
print(recall_score(result_chain_older_metadata.ground_truth, result_chain_older_metadata.numeric_prediction))

0.5216494845360825
0.9219858156028369
0.35755813953488375
0.37037037037037035
0.9219858156028369


In [15]:
print(result_chain_older.shape[0])
print(result_chain_older[result_chain_older.probability > 90].correct.mean())
print(result_chain_older[result_chain_older.probability > 90].shape[0])
print(result_chain_older[result_chain_older.probability <= 90].correct.mean())

485
0.8292682926829268
41
0.49324324324324326


In [17]:
result_chain = l.evaluate(l.zero_shot_predictor, 0, l.d.shape[0])

At data 54, error: The server is overloaded or not ready yet.
At data 350, error: The server is overloaded or not ready yet.


In [18]:
result_chain = pd.DataFrame(result_chain)
result_chain.numeric_prediction.value_counts()

0    363
1    122
Name: numeric_prediction, dtype: int64

In [19]:
result_chain[['probability', 'numeric_prediction']].value_counts()

probability  numeric_prediction
99           0                     347
95           1                      68
90           1                      51
95           0                      12
100          0                       4
99           1                       2
80           1                       1
dtype: int64

In [20]:
print(result_chain.text_response[6])


ANSWER: YES. EXPLANATION: Alkermes Plc is a biopharmaceutical company that develops and commercializes products designed to address unmet medical needs of patients in major therapeutic areas, including addiction and schizophrenia. This bill requires the Centers for Medicare & Medicaid Services (CMS) to negotiate with pharmaceutical companies regarding prices for drugs covered under the Medicare prescription drug benefit, which could potentially impact Alkermes Plc's products. CONFIDENCE: 95


In [21]:
print(result_chain.correct.mean())
print(result_chain[result_chain.ground_truth == 1].correct.mean())
print(result_chain[result_chain.ground_truth == 0].correct.mean())
result_chain_metadata = pd.concat([l.d, result_chain], axis=1)
print(precision_score(result_chain_metadata.ground_truth, result_chain_metadata.numeric_prediction))
print(recall_score(result_chain_metadata.ground_truth, result_chain_metadata.numeric_prediction))

0.7505154639175258
0.5035460992907801
0.8517441860465116
0.5819672131147541
0.5035460992907801


In [22]:
letters = list()
for i in result_chain[result_chain.numeric_prediction == 1].index[:5]:
    letters.append(l.zero_shot_letter_generator(i))

In [23]:
print(letters[0])


Dear [Sponsor of the Bill],

I am writing on behalf of Alkermes Plc, a fully integrated, global biopharmaceutical company that applies its scientific expertise and proprietary technologies to research, develop and commercialize pharmaceutical products that are designed to address unmet medical needs of patients in major therapeutic areas.

We are writing to express our support for the Medicare Negotiation and Competitive Licensing Act of 2019. We believe that this bill is an important step in ensuring that Medicare beneficiaries have access to the medications they need at a price they can afford.

We are particularly supportive of the provisions in the bill that would require the Centers for Medicare & Medicaid Services (CMS) to negotiate with pharmaceutical companies regarding prices for drugs covered under the Medicare prescription drug benefit. We believe that this will help to ensure that the prices of these drugs are fair and reasonable.

We are also supportive of the provisions 

In [24]:
print(result_chain.shape[0])
print(result_chain[result_chain.probability > 90].correct.mean())
print(result_chain[result_chain.probability > 90].shape[0])
print(result_chain[result_chain.probability <= 90].correct.mean())

485
0.789838337182448
433
0.4230769230769231
